# Nettoyeur

Fichiers de sortie produits par le notebook :
- org.txt -> point de référence pour comparer la qualité du nettoyage
- clean_{date}.txt -> fichier texte de après nettoyage avec date
- PUIS au choix :
    - clean_w_names_{date}.json -> **fichier json destiné à être repris dans la suite des analyses**
    - clean_w_names_{date}.csv -> **fichier csv destiné à être repris dans la suite des analyses**

> Pour reconvertir le JSON en dataframe panda, utiliser la commande suivante au début du notebook :

df = pd.read_json('NOM_DU_FICHIER.json', orient='index')

> Pour explorer le JSON -> télécharger puis ouvrir avec Firefox (les autres navigateurs ne font pas la mise en forme)

In [1]:
import pandas as pd
import regex as re
from datetime import date
import time

In [2]:
# Création d'un dataframe à partir du tableau d'origine
df = pd.read_csv("data/20221010_jds_CR.csv", keep_default_na=False)  # dernier paramètre : éviter les NaN (considérés comme float et non  str)

In [3]:
# On ne laisse que les colonnes "area_title" et "area_text"
df = pd.DataFrame(df, columns=['area_title', 'area_text'])

# Regroupement des textes par titres. Certains titres étaient présents plusieurs fois : les texte étaient en plusieurs morceaux
df = df.groupby('area_title', as_index=False).agg({'area_title' : 'first', 'area_text' : ' '.join})

In [4]:
# Constitution d'un fichier texte contenant les textes originaux pour comparaison
# Réinitialisation du fichier
open("results/org.txt", "w").close()

with open('results/org.txt', 'a') as org:
    org.write('ORIGINAL' + 3*'\n')

    for i in range(len(df.index)):
        org.write('TITRE\n' + df.loc[i].at['area_title'] + '\n')
        org.write('TEXTE\n' + df.loc[i].at['area_text'] + 2*'\n')


In [5]:
# Traitements de nettoyage
liste_remplacements = [
    ["<.{1}>|</.{1}>", ""],
    ["¬ ", ""],
    [" , ", ", "],
    [":", " : "],
    [" {2,}", " "],
    [" \.", "."],
    ["' ", "'"],
    ["(?<=[0-9])[aoO](?=[0-9])|(?<=[0-9])[aoO] ", "0"],
    ["(?<=[0-9])i(?=[0-9])| i(?=[0-9])|(?<=[0-9])i ", "1"],
    ["i (?=[0-9])", "1"],
    ["^ ", ""],
    ["- ", ""],
    ["\n", ""],
    ["\t", ""],
    ["(?<=[a-zA-Z0-9])à ", " à "],
    [" à(?=[a-zA-Z0-9])", " à "],
    [" en(?=[0-9])", " en "],
    [" En(?=[0-9])", " En "],
    [" de(?=[0-9])", " de "],
    [" De(?=[0-9])", " De "],
    [" dès(?=[0-9])", " dès "],
    [" Dès(?=[0-9])", " Dès "],
    [" de, s ", " des "],
    [" de,s ", " des "],
    [" à ia ", " à la "],
    [" dansle ", " dans le "],
    [" dansla ", " dans la "],
    [" queM. ", " que M. "],
    [" etd", " et d"],
    [" etD", " et D"],
    ["~", ""],
    [" M, ", " M. "],
    [" parM\.", " par M."],
    [" deM\.", " de M."],
    ["M\.(?=[A-Za-z])", "M. "]
    ]

# Conversion en dataframe
df_remplacements = pd.DataFrame(liste_remplacements, columns = ['org', 'new'])

In [6]:
# Remplacement au sein du dataframe original
for i in range(len(df_remplacements.index)):
    df = df.replace(
        {"area_title": df_remplacements["org"][i]},
        {"area_title": df_remplacements["new"][i]},
        regex=True,
    )
    df = df.replace(
        {"area_text": df_remplacements["org"][i]},
        {"area_text": df_remplacements["new"][i]},
        regex=True,
    )

# Boucle de récupération du nom de l'auteur du CR via la présence de sa signature à la fin du plein texte
names = []
for i in range(len(df.index)):
    text = df.loc[i].at['area_text']
    end = text[-40:-2]
    end = re.sub('^.*?[.!?](?:\s|$)(?!.*\))', '', end)
    if len(end) > 25:
        end = re.sub('\.', '. ', end)
        end = re.sub('^.*?[.!?](?:\s|$)(?!.*\))', '', end)
    end = re.sub('{|}|<|>|;|:|\?|«|»|!|\(|_|[0-9]', '', end)
    end = re.sub('^\.', '', end)
    end = re.sub('^—', '', end)
    end = re.sub('^ ', '', end)

    names.append(end)

df['extracted_names'] = names

In [8]:
# Mise au point du nom des fichiers
date = time.strftime("%d-%m-%y")
nom_fichier = f"results/clean_w_names_{date}"

In [9]:
# Constituton du txt clean complet
# Ouverture puis fermeture pour recréer un fichier vierge si plusieurs exécutions le même jour
open(f"{nom_fichier}.txt", "w").close()

# Remplissage du fichier clean complet
with open(f"{nom_fichier}.txt", 'a') as clean:

    # Inscription de la date au début
    clean.write('DATE : ' + date + 3*'\n')

    # Peuplement
    for i in range(len(df.index)):
        clean.write('CR n° ' + str(i) + '\n')
        clean.write('TITRE\n' + df.loc[i].at['area_title'] + '\n')
        clean.write('TEXTE\n' + df.loc[i].at['area_text'] + 2*'\n')

In [10]:
# # Constituton du txt par échantillons
# # Nombre d'échantillons souhaités
# echantillons = 40

# # Ouverture puis fermeture pour recréer un fichier vierge si plusieurs exécutions le même jour
# open(f"{nom_fichier}_{echantillons}_samples.txt", "w").close()

# # Remplissage du fichier clean complet
# with open(f"{nom_fichier}_{echantillons}_samples.txt", 'a') as clean:

#     # Inscription de la date au début
#     # clean.write(f'DATE : {date}\nÉchantillons : {echantillons}' + 3*'\n')

#     # Peuplement
#     for i in range(0, len(df.index), int(len(df.index) / (echantillons - 1))):
#         # clean.write('CR n° ' + str(i) + '\n')
#         # clean.write('TITRE\n' + df.loc[i].at['area_title'] + '\n')
#         clean.write(df.loc[i].at['area_text'] + 2*'\n')

In [11]:
# Export en en fichier JSON
df.to_json(f'{nom_fichier}.json', orient='index')

# Export en CSV
df.to_csv(f'{nom_fichier}.csv')
